In [ ]:
## Name: Jens and Sarah Mehler
## Date: March, 17th 2020
## Create Wave File of Chirp Trains



In [ ]:
# Setup Notebook
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
from scipy.signal import chirp


In [ ]:
# Setup Parameters
DEBUG = False
#___|Center Freq = fc |______________________________|Center Freq = fc |_________________
#   [<------- Pulse Repetition Interval = PRI   ---->]
#   [<--Pulse Width-->]
amplitude = np.iinfo(np.int16).max
A = amplitude

# This is the setup for the chirp. It will do a chirp from fmin to fmax for however long PW is. If fmax is
# too high then you might have to reduce your sampling rate. Number of periods is how many periods there
# would be for fmax so you need a lot for your fmin to have good resolution.

#Pick a center frequency
center_freq = 5e3

# Set the sweep to be 1/4 as wide as fc according to random research paper on sonar. 
# http://www.htisonar.com/publications/FMSlideChirp.pdf
fmin = center_freq - center_freq/8
fmax = center_freq + center_freq/8

# fc is now set to fmax so the old logic on how to set sampling for a single tone
# sinusoid will be applied to our highest frequency
fc = fmax # 4kHz kilohertz


# What resolution do you want in meters
res_meters = 1
num_periods = 1
Tc= res_meters/343
PW = Tc*num_periods

# Whats your desired Unambiguos Range in meters?
UR=40 # meters
PRI=round((2*UR+343*PW)/343,3)


print('='*20)
print("Time and Frequency")
print('PRI:{} s, PW {}s, Fc {} Hz, Tc {} s'.format(PRI, PW, fc, Tc))
print('\n')

# Spatial information 
vp = 343 # m/s speed of sound 
PRI_x = PRI * vp
PW_x = PW * vp
kc = fc/vp
Lc = vp/fc

print('='*20)
print("Space and Time")
print('PRI:{:.02f} m, PW {:.02f} m, kc {:.02f} cycles/meter, lambda {:.02f} m'
      .format(PRI_x, PW_x, kc, Lc))


In [ ]:
# Making the data pretty
import pandas as pd
data1 = [[PRI, PW, fc, Tc],[PRI, PW*1e3, fc/1e3, Tc*1e6]]
pd.DataFrame(data1[1:], columns=["PRI (s)", "PW(ms)", "fc (kHz)", "Tc ($\mu$s)"])





In [ ]:
data2 = [[PRI_x, PW_x, kc, Lc],[PRI_x, PW_x, kc, Lc]]
pd.DataFrame(data2[1:], columns=["PRI (m)", "PW(m)", "kc (cycles/meter)", "$\lambda$ (m)"])


## Making the unit waveform

In [ ]:
# samples per period 
#samps_per_period = 10
FS_MAX = 100000 # maximum sample rate for audio transmitter

# create the time step for sampling 
fmax = fc 
Tmax = 1/fmax
dt = Tmax/20

# Optional: create a waveform with dt/[2,3,4,5,6,7,8,9,10,15,20] and plot the FFTs of each 
# You should see what happens to the energy when the signals are reproduced cleanly. 

# calculate required sample rate
fs = 1/dt
derp = fs < FS_MAX
print('Desired Sample Rate : {} samples/sec'.format(fs))
print('Sample Rate Check: is {} less than {} == {}'.format(fs, FS_MAX, derp))

if derp == False:
    print('\n ***** Desired Sample Rate too high! ***** ')
else:
    print(' Desired Sample Rate usable')
    

In [ ]:
# create the time series array
t_unit = PRI
t_vector = np.arange(0,t_unit,dt)
print('Samples in unit vector {}'.format(len(t_vector)))
plt.plot(t_vector)
plt.xlabel('index')
plt.ylabel('seconds')
plt.title('Time Series Vector')

# checking to see if we are creating it correctly

In [ ]:
print(PW)

In [ ]:
## Making PW the iteration way
# now we are going to create a mask for bits that will be on or off
# create a single pulse train
# 111111000000000


mask = np.zeros_like(t_vector)
sample = t_vector[0]
idx = 1 
while sample < PW:
    mask[idx] = 1
    idx = idx+1
    sample = t_vector[idx]

plt.plot(t_vector[:len(mask)],mask)
plt.xlabel('seconds')
plt.ylabel('True/False')
plt.title('Mask for Pulse Train PW = {}s'.format(PW))

In [ ]:
## Create mask vector method
# figure out how many samples are in the pulse
PWidx = np.int(PW/dt)

mask_vector = np.zeros_like(t_vector)
mask_vector[:PWidx] = 1 

mask_vector[0]=0 # makes the pulse nice. 

plt.plot(t_vector,mask_vector)
plt.xlabel('seconds')
plt.ylabel('True/False')
plt.title('Mask for Pulse Train PW = {}s'.format(PW))

In [ ]:
# Create the sine wave
amplitude = np.iinfo(np.int16).max
A =amplitude
print(fc)
#data = amplitude * np.sin(2. * np.pi * freq * t)
t = np.arange(0,PW,dt)
signal = A*chirp(t_vector,fmin,PW,fmax,'linear',phi=-90)
plt.figure()
plt.plot(signal[0:len(t)])

plt.figure()
plt.plot(t_vector[0:PWidx//200], signal[0:PWidx//200],'o-')
#plt.plot(t_vector[:PWidx], signal[:PWidx]) # zeros are optional 
plt.title('Fc {} kHz    For the duration of the pulse'.format(fc/1e3))
plt.xlabel('time (s)')
plt.ylabel('Amplitude')
signal[:20]

In [ ]:
# now combine everything
signal = signal * mask
plt.plot(t_vector,signal)
plt.title('Pulse Train Unit')
plt.xlabel('time (s)')
plt.ylabel('Amplitude')

In [ ]:
# Now we make a long time series of these
t_max = 60+.01 # 5 min the +3 is to show rounding

periods_to_copy = t_max / PRI
print('Periods to copy {}'.format(periods_to_copy))
periods_to_copy = np.int(np.ceil(periods_to_copy))
print('Periods to copy {}'.format(periods_to_copy))

unit_signal = signal
signal = None
signal = unit_signal
for idx in range(0,periods_to_copy):
    signal=np.concatenate((signal, unit_signal), axis=None)

t_max_idx = len(signal)
t_vector = np.arange(0,len(signal)*dt,dt)    
if DEBUG == True:
    plt.plot(t_vector, signal, 'xkcd:mango')
    plt.title('Pulse Train')
    plt.xlabel('Seconds')
    plt.ylabel('Amplitude')
    
print(len(signal)*dt)

In [ ]:
plt.plot(signal[:72000])

In [ ]:

from scipy.io.wavfile import write
samplerate = 44100; 
'''
#freq = 100
t = np.linspace(0., 1., samplerate)
amplitude = np.iinfo(np.int16).max
print(amplitude)
data = amplitude * np.sin(2. * np.pi * freq * t)
write("example.wav", samplerate, data)
print(data[:20])
'''

In [ ]:

# Write out the waveform
print(signal[0:20])
samplerate = np.int(fs)
print(fs)
write("ChirpTrain.wav", samplerate, signal)
print(signal[0:20])

#### 